In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec, FastText
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

Загрузка ресурсов NLTK для стоп-слов и токенизаци

In [40]:

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Загрузка данных

In [41]:
data = pd.read_csv("spam_or_not_spam.csv", header=None, names=['email', 'label'])

Замена,Токенизация текста,Удаление стоп-слов и пунктуации

In [42]:
def preprocess_text(text):
    if isinstance(text, str):  # Проверяем, является ли значение строкой
        # Замена "NUMBER" на "number" и "URL" на "url"
        text = text.replace("NUMBER", "number")
        text = text.replace("URL", "url")

        # Токенизация текста
        tokens = word_tokenize(text.lower())

        # Удаление стоп-слов и пунктуации
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

        return tokens
    else:
        return []  # Возвращаем пустой список для значений, не являющихся строками

Применение предобработки к столбцу с электронными письмами

In [43]:
data['clean_text'] = data['email'].apply(preprocess_text)

Разделение данных на обучающий и тестовый наборы

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.2, random_state=42)

Обучение модели

In [45]:
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, sg=1, min_count=1)
ft_model = FastText(sentences=X_train, vector_size=100, window=5, min_count=1, sg=1)

Оценка векторных представлений с использованием методов most_similar и doesnt_match

In [46]:
def evaluate_embeddings(model):
    print("Word Similarity:")
    print(model.wv.most_similar("email"))
    print(model.wv.most_similar("spam"))
    try:
        print(model.wv.most_similar("not"))
    except KeyError:
        print("'not' not present in vocabulary")

    print("\nDoesn't Match:")
    print(model.wv.doesnt_match(["email", "message", "phone"]))
    print(model.wv.doesnt_match(["spam", "junk", "ham"]))
    try:
        print(model.wv.doesnt_match(["not", "no", "yes"]))
    except KeyError:
        print("'not' not present in vocabulary")

Оценка векторных представлений для модели Word2Vec

In [47]:
print("Word2Vec Evaluation:")
evaluate_embeddings(w2v_model)

Word2Vec Evaluation:
Word Similarity:
[('watching', 0.7851579785346985), ('assignee', 0.7700648307800293), ('sponsored', 0.7651469707489014), ('dump', 0.764888346195221), ('blank', 0.7553474307060242), ('sincerely', 0.7539013624191284), ('opt', 0.749565839767456), ('opted', 0.7454697489738464), ('replying', 0.7443695068359375), ('promotional', 0.7437946200370789)]
[('ham', 0.8581007122993469), ('corpus', 0.8067672848701477), ('nonspam', 0.784724235534668), ('filtering', 0.7835919857025146), ('headers', 0.7724446654319763), ('setnumber', 0.7721927165985107), ('score', 0.7707676887512207), ('trained', 0.769098162651062), ('training', 0.7622032165527344), ('filters', 0.761142373085022)]
'not' not present in vocabulary

Doesn't Match:
message
junk
yes


Оценка векторных представлений для модели fastText

In [48]:
print("\nFastText Evaluation:")
evaluate_embeddings(ft_model)


FastText Evaluation:
Word Similarity:
[('emails', 0.9407628774642944), ('bmail', 0.9314588308334351), ('emailers', 0.9310613870620728), ('emailed', 0.9287028312683105), ('orig_email', 0.9281359910964966), ('omail', 0.9236548542976379), ('webmail', 0.9187140464782715), ('nomoremail', 0.9185559749603271), ('qmail', 0.9185280203819275), ('rmail', 0.917107343673706)]
[('spamm', 0.9552136659622192), ('spammy', 0.9550026059150696), ('spamcop', 0.9300490021705627), ('spaß', 0.9292759299278259), ('spamtrap', 0.9282188415527344), ('spamc', 0.9254674315452576), ('spamhams', 0.9242157340049744), ('nospam', 0.921402096748352), ('spamd', 0.9170072674751282), ('nonspam', 0.9168686270713806)]
[('logwatch', 0.9762375950813293), ('butt', 0.9741703271865845), ('login', 0.9731600284576416), ('autopsy', 0.9726884961128235), ('snot', 0.9720602631568909), ('_not_', 0.9702712893486023), ('autopc', 0.9692462086677551), ('notch', 0.9687154293060303), ('whit', 0.9665449261665344), ('patch', 0.9655031561851501)

Преобразование текста в векторы с использованием Word2Vec

In [49]:
def text_to_vectors(model, text):
    vectors = [model.wv[word] for word in text if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

Преобразование текста в векторы для обучающего и тестового наборов

In [50]:
X_train_w2v = np.array([text_to_vectors(w2v_model, text) for text in X_train])
X_test_w2v = np.array([text_to_vectors(w2v_model, text) for text in X_test])

X_train_ft = np.array([text_to_vectors(ft_model, text) for text in X_train])
X_test_ft = np.array([text_to_vectors(ft_model, text) for text in X_test])

Инициализация и обучение модели логистической регрессии на векторах

In [51]:
lr_model_w2v = LogisticRegression()
lr_model_w2v.fit(X_train_w2v, y_train)

lr_model_ft = LogisticRegression()
lr_model_ft.fit(X_train_ft, y_train)

LogisticRegression()

Оценка точности модели на тестовом наборе данных

In [52]:
accuracy_w2v = lr_model_w2v.score(X_test_w2v, y_test)
print("\nLogistic Regression Accuracy (Word2Vec):", accuracy_w2v)

accuracy_ft = lr_model_ft.score(X_test_ft, y_test)
print("Logistic Regression Accuracy (FastText):", accuracy_ft)


Logistic Regression Accuracy (Word2Vec): 0.9717138103161398
Logistic Regression Accuracy (FastText): 0.9717138103161398


Из полученных данных следует следующий вывод:

Word2Vec Evaluation:

Модель Word2Vec дает хорошие результаты в оценке векторных представлений слов и находит семантически близкие слова, однако некоторые слова, такие как 'not' и 'no', отсутствуют в словаре, что может быть проблемой при работе с отрицаниями или отрицательными контекстами.

FastText Evaluation:

Модель FastText хорошо работает с векторными представлениями слов, даже при наличии опечаток или нестандартных символов благодаря использованию субслов. Метод most_similar находит схожие слова даже с опечатками или похожими субсловами.

Логистическая регрессия:

Модель логистической регрессии на векторных представлениях слов (Word2Vec и FastText) дает точность в 97-98% при классификации спама, что подтверждает эффективность выбранных методов предобработки и векторизации текста.

